In [123]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import Lasso, LinearRegression, Ridge, ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import json
from tqdm.notebook import tqdm
from scipy.stats import skew

In [2]:
with open("feature_dict.json", "r") as json_file:
    feature_dict = json.load(json_file)

In [3]:
df = pd.read_csv('train_prepared_for_cv.csv')

In [4]:
df.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice          int64
Length: 80, dtype: object

In [5]:
sum(df.isna().sum())

357

In [6]:
X = df.drop(columns=['Id', 'SalePrice'])
y = df['SalePrice']

In [7]:
numeric_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]
numeric_cols.remove("MSSubClass")
categorical_cols = [col for col in X.columns if col not in numeric_cols]
ordinal_encode_cols = ["ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "BsmtExposure", "HeatingQC", "KitchenQual", "FireplaceQu", "GarageQual", "GarageCond", "PoolQC", "Street", "Alley", "PavedDrive"]
label_encode_cols = [col for col in categorical_cols if col not in ordinal_encode_cols]
ohe_encode_cols = label_encode_cols

In [8]:
print(numeric_cols)

['LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold']


In [9]:
print(categorical_cols)

['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [10]:
print(ordinal_encode_cols)

['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC', 'Street', 'Alley', 'PavedDrive']


In [11]:
print(label_encode_cols)

['MSSubClass', 'MSZoning', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtFinType1', 'Heating', 'CentralAir', 'Electrical', 'Functional', 'GarageType', 'GarageFinish', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [12]:
print(len(categorical_cols))
print(len(ordinal_encode_cols), len(label_encode_cols), len(ohe_encode_cols))
print(len(ordinal_encode_cols) + len(label_encode_cols))
print(len(numeric_cols))
print(len(numeric_cols) + len(categorical_cols))

43
14 29 29
43
35
78


In [13]:
ordinal_mappings = {}

for col in ordinal_encode_cols:
    if col in X.columns and col in feature_dict:
        all_values = list(feature_dict[col]["values"].keys())  # Extract and preserve the order of values from the dict
        mapping = {category: rank for rank, category in enumerate(reversed(all_values), start=1)}  # Encode in the dict order
        mapping["No_item"] = -1
        if "NA" in mapping.keys():
            del mapping["NA"]
        ordinal_mappings[col] = mapping
        
def safe_map(value, mapping):
    if value not in mapping:
        raise ValueError(f"Value '{value}' not found in the mapping!")
    return mapping[value]

def ordinal_encode_column(X, mappings):
    encoded = X.copy()
    for col in X.columns:
        if col in mappings:
            encoded[col] = X.apply(lambda x: safe_map(x, mappings[col]))
    return encoded

print(ordinal_mappings)

{'ExterQual': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, 'No_item': -1}, 'ExterCond': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, 'No_item': -1}, 'BsmtQual': {'Po': 2, 'Fa': 3, 'TA': 4, 'Gd': 5, 'Ex': 6, 'No_item': -1}, 'BsmtCond': {'Po': 2, 'Fa': 3, 'TA': 4, 'Gd': 5, 'Ex': 6, 'No_item': -1}, 'BsmtExposure': {'No': 2, 'Mn': 3, 'Av': 4, 'Gd': 5, 'No_item': -1}, 'HeatingQC': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, 'No_item': -1}, 'KitchenQual': {'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, 'No_item': -1}, 'FireplaceQu': {'Po': 2, 'Fa': 3, 'TA': 4, 'Gd': 5, 'Ex': 6, 'No_item': -1}, 'GarageQual': {'Po': 2, 'Fa': 3, 'TA': 4, 'Gd': 5, 'Ex': 6, 'No_item': -1}, 'GarageCond': {'Po': 2, 'Fa': 3, 'TA': 4, 'Gd': 5, 'Ex': 6, 'No_item': -1}, 'PoolQC': {'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, 'No_item': -1}, 'Street': {'Grvl': 1, 'Pave': 2, 'No_item': -1}, 'Alley': {'Grvl': 2, 'Pave': 3, 'No_item': -1}, 'PavedDrive': {'N': 1, 'P': 2, 'Y': 3, 'No_item': -1}}


In [14]:
label_mappings = {}

for col in label_encode_cols:
    if col in X.columns and col in feature_dict:
        all_values = list(feature_dict[col]["values"].keys())  # Extract and preserve the order of values from the dict
        mapping = {category: rank for rank, category in enumerate(reversed(all_values), start=1)}  # Encode in the dict order
        mapping["No_item"] = -1
        if "NA" in mapping.keys():
            del mapping["NA"]
        label_mappings[col] = mapping
        
def safe_map(value, mapping):
    if value not in mapping:
        raise ValueError(f"Value '{value}' not found in the mapping!")
    return mapping[value]

def label_encode_column(X, mappings):
    encoded = X.copy()
    for col in X.columns:
        if col in mappings:
            encoded[col] = df[col].apply(lambda x: safe_map(x, mappings[col]))
    return encoded

print(label_mappings)

{'MSSubClass': {'190': 1, '180': 2, '160': 3, '150': 4, '120': 5, '90': 6, '85': 7, '80': 8, '75': 9, '70': 10, '60': 11, '50': 12, '45': 13, '40': 14, '30': 15, '20': 16, 'No_item': -1}, 'MSZoning': {'RM': 1, 'RP': 2, 'RL': 3, 'RH': 4, 'I': 5, 'FV': 6, 'C': 7, 'A': 8, 'No_item': -1}, 'LotShape': {'IR3': 1, 'IR2': 2, 'IR1': 3, 'Reg': 4, 'No_item': -1}, 'LandContour': {'Low': 1, 'HLS': 2, 'Bnk': 3, 'Lvl': 4, 'No_item': -1}, 'Utilities': {'ELO': 1, 'NoSeWa': 2, 'NoSewr': 3, 'AllPub': 4, 'No_item': -1}, 'LotConfig': {'FR3': 1, 'FR2': 2, 'CulDSac': 3, 'Corner': 4, 'Inside': 5, 'No_item': -1}, 'LandSlope': {'Sev': 1, 'Mod': 2, 'Gtl': 3, 'No_item': -1}, 'Neighborhood': {'Veenker': 1, 'Timber': 2, 'StoneBr': 3, 'Somerst': 4, 'SawyerW': 5, 'Sawyer': 6, 'SWISU': 7, 'OldTown': 8, 'NWAmes': 9, 'NridgHt': 10, 'NPkVill': 11, 'NoRidge': 12, 'Names': 13, 'Mitchel': 14, 'MeadowV': 15, 'IDOTRR': 16, 'Gilbert': 17, 'Edwards': 18, 'Crawfor': 19, 'CollgCr': 20, 'ClearCr': 21, 'BrkSide': 22, 'BrDale': 23, 

In [151]:
class DataFramePreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, numeric_cols, ordinal_cols, ohe_cols, label_cols, ordinal_mappings, label_mappings, scale_features_except_ohe=False):
        self.numeric_cols = numeric_cols
        self.ordinal_cols = ordinal_cols
        self.ohe_cols = ohe_cols
        self.label_cols = label_cols
        self.ordinal_mappings = ordinal_mappings
        self.label_mappings = label_mappings
        self.ohe_columns_names = None
        self.medians = {}
        self.modes = {}
        self.scaler = StandardScaler()
        self.scale_features_except_ohe = scale_features_except_ohe

    def fit(self, X, y=None):
        self.medians = X[self.numeric_cols].median()
        self.modes = X[self.ordinal_cols + self.ohe_cols + self.label_cols].mode().iloc[0]
        if self.ohe_cols:
            self.ohe_columns_names = []
            for col in self.ohe_cols:
                if col in feature_dict:
                    categories = feature_dict[col]["values"].keys()
                    for category in categories:
                        self.ohe_columns_names.append(f"{col}_{category}")
                else:
                    raise KeyError(f"Feature {col} not found in feature dictionary")
        return self

    def transform(self, X):
        X = X.copy()
        # filling NA
        for col in self.numeric_cols:
            X[col] = X[col].fillna(self.medians[col])
        
        # skewness = X[numeric_cols].apply(lambda x: skew(x))
        # skewness = skewness[abs(skewness) > 0.5]
        # skewed_features = skewness.index
        # X[skewed_features] = np.log1p(X[skewed_features])
    
        if X.columns.duplicated().any():
                raise ValueError(f"Duplicate column names found: {X.columns[X.columns.duplicated()].tolist()}")

        for col in self.ordinal_cols + self.ohe_cols + self.label_cols:
            X[col] = X[col].fillna(self.modes[col])
        
        for col in self.ordinal_cols:
            if col in self.ordinal_mappings:
                X[col] = X[col].map(self.ordinal_mappings[col]).fillna(-1)
            else:
                raise KeyError
        
        for col in self.label_cols:
            if col in self.label_mappings:
                X[col] = X[col].map(self.label_mappings[col]).fillna(-1)
            else:
                raise KeyError
        
        if self.scale_features_except_ohe == True:
            X[self.label_cols + self.ordinal_cols +self.numeric_cols] = self.scaler.fit_transform(X[self.label_cols + self.ordinal_cols + self.numeric_cols])

        
        if self.ohe_cols:
            ohe_encoded = pd.DataFrame(0, index=X.index, columns=self.ohe_columns_names)
            for col in self.ohe_cols:
                if col in feature_dict:
                    categories = feature_dict[col]["values"].keys()
                    for category in categories:
                        ohe_col_name = f"{col}_{category}"
                        ohe_encoded[ohe_col_name] = (X[col] == category).astype(int)
                else:
                    raise KeyError(f"Feature {col} not found in feature dictionary")
            X = X.drop(columns=self.ohe_cols)
            X = pd.concat([X, ohe_encoded], axis=1)
            
        return X

In [129]:
preprocessor = DataFramePreprocessor(
    numeric_cols=numeric_cols,
    ordinal_cols=ordinal_encode_cols,
    ohe_cols=ohe_encode_cols,
    label_cols=[],
    ordinal_mappings=ordinal_mappings,
    label_mappings=label_mappings,
    # scale_features_except_ohe=True
)

In [130]:
# testing preprocessing
processed_data = preprocessor.fit_transform(X)

In [131]:
# These feature have skewness > 0.5
# skewed_cols = ['LotFrontage', 'LotArea', 'OverallCond', 'YearBuilt', 'MasVnrArea',
#        'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF',
#        'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'HalfBath',
#        'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt',
#        'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
#        'ScreenPorch', 'PoolArea', 'MiscVal']

In [18]:
processed_data

,LotFrontage,LotArea,Street,Alley,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,...,SaleType_ConLw,SaleType_ConLI,SaleType_ConLD,SaleType_Oth,SaleCondition_Normal,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Partial
0,65.0,8450,2,-1,7,5,2003,2003,196.0,4,...,0,0,0,0,1,0,0,0,0,0
1,80.0,9600,2,-1,6,8,1976,1976,0.0,3,...,0,0,0,0,1,0,0,0,0,0
2,68.0,11250,2,-1,7,5,2001,2002,162.0,4,...,0,0,0,0,1,0,0,0,0,0
3,60.0,9550,2,-1,7,5,1915,1970,0.0,3,...,0,0,0,0,0,1,0,0,0,0
4,84.0,14260,2,-1,8,5,2000,2000,350.0,4,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,62.0,7917,2,-1,6,5,1999,2000,0.0,3,...,0,0,0,0,1,0,0,0,0,0
1452,85.0,13175,2,-1,6,6,1978,1988,119.0,3,...,0,0,0,0,1,0,0,0,0,0
1453,66.0,9042,2,-1,7,9,1941,2006,0.0,5,...,0,0,0,0,1,0,0,0,0,0
1454,68.0,9717,2,-1,5,6,1950,1996,0.0,3,...,0,0,0,0,1,0,0,0,0,0


Playing with models

In [137]:
def run_model(model_class, param_grid, preprocessor, X, y):
    
    kf = KFold(n_splits=9, shuffle=True, random_state=42)
    
    r2 = {"train": [], "test": []}
    mae = {"train": [], "test": []}
    best_params = []
    feature_importances = pd.DataFrame(index=X.columns)

    for fold, (train_index, test_index) in tqdm(enumerate(kf.split(X), 1)):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        preprocessor.fit(X_train)
        X_train_preprocessed = preprocessor.transform(X_train)
        X_test_preprocessed = preprocessor.transform(X_test)
        
        model = model_class()
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_absolute_error', n_jobs=-1)
        grid_search.fit(X_train_preprocessed, y_train)
        
        best_params.append(grid_search.best_params_)
        
        model = model_class(**grid_search.best_params_)
        model.fit(X_train_preprocessed, y_train)
        
        y_train_pred = model.predict(X_train_preprocessed)
        y_test_pred = model.predict(X_test_preprocessed)
        
        train_mae = mean_absolute_error(y_train, y_train_pred)
        test_mae = mean_absolute_error(y_test, y_test_pred)
        train_r2 = r2_score(y_train, y_train_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        
        r2["train"].append(train_r2)
        r2["test"].append(test_r2)
        mae["train"].append(train_mae)
        mae["test"].append(test_mae)
        
        if hasattr(model, "coef_"):
            try:
                fold_importances = pd.Series(np.abs(model.coef_), index=X.columns)
            except:
                fold_importances = pd.Series(np.abs(model.coef_), index=X_train_preprocessed.columns)
        elif hasattr(model, "feature_importances_"):
            try:
                fold_importances = pd.Series(model.feature_importances_, index=X.columns)
            except:
                fold_importances = pd.Series(model.feature_importances_, index=X_train_preprocessed.columns)
        else:
            raise AttributeError(f"The model {model_class.__name__} does not support feature importance extraction.")
        
        feature_importances = pd.concat([feature_importances, fold_importances], axis=1)
    
    feature_importances['mean_importance'] = feature_importances.mean(axis=1)
    feature_importances = feature_importances[['mean_importance']].sort_values(by='mean_importance', ascending=False)
    
    return pd.DataFrame(r2), pd.DataFrame(mae), feature_importances, best_params

In [155]:
# label without scaling

In [157]:
preprocessor = DataFramePreprocessor(
    numeric_cols=numeric_cols,
    ordinal_cols=ordinal_encode_cols,
    ohe_cols=[],
    label_cols=label_encode_cols,
    ordinal_mappings=ordinal_mappings,
    label_mappings=label_mappings,
    scale_features_except_ohe=False
)
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
r2, mae, feature_importances, best_params = run_model(model_class=Lasso, param_grid=param_grid, preprocessor=preprocessor, X=X, y=y)
print("MAE:\n", mae.mean())
print("-" * 50)
print("R2:\n", r2.mean())
print("-" * 50)
print("Best parameters:\n", best_params)
feature_importances

0it [00:00, ?it/s]

MAE:
 train    17337.069397
test     18225.590336
dtype: float64
--------------------------------------------------
R2:
 train    0.893988
test     0.879915
dtype: float64
--------------------------------------------------
Best parameters:
 [{'alpha': 1000}, {'alpha': 1000}, {'alpha': 1000}, {'alpha': 1000}, {'alpha': 100}, {'alpha': 1000}, {'alpha': 1000}, {'alpha': 100}, {'alpha': 1000}]


,mean_importance
OverallQual,11238.493998
ExterQual,7932.347160
BedroomAbvGr,5640.297712
KitchenQual,5215.830560
BsmtCond,4563.945384
...,...
FullBath,0.000000
LotConfig,0.000000
Utilities,0.000000
Street,0.000000


In [160]:
param_grid = {}
r2, mae, feature_importances, best_params = run_model(model_class=LinearRegression, param_grid=param_grid, preprocessor=preprocessor, X=X, y=y)
print("MAE:\n", mae.mean())
print("-" * 50)
print("R2:\n", r2.mean())
print("-" * 50)
print("Best parameters:\n", best_params)
feature_importances

0it [00:00, ?it/s]

MAE:
 train    17052.607791
test     18427.241993
dtype: float64
--------------------------------------------------
R2:
 train    0.901383
test     0.882449
dtype: float64
--------------------------------------------------
Best parameters:
 [{}, {}, {}, {}, {}, {}, {}, {}, {}]


,mean_importance
Street,2.104157e+04
KitchenAbvGr,1.943977e+04
Utilities,1.930224e+04
PoolQC,1.482452e+04
OverallQual,1.027386e+04
...,...
BsmtFinSF2,5.175923e+00
BsmtUnfSF,1.347718e+00
MiscVal,6.110004e-01
LotArea,5.334086e-01


In [161]:
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'l1_ratio': [0.1, 0.5, 0.7, 0.9, 1.0]      
}
r2, mae, feature_importances, best_params = run_model(model_class=ElasticNet, param_grid=param_grid, preprocessor=preprocessor, X=X, y=y)
print("MAE:\n", mae.mean())
print("-" * 50)
print("R2:\n", r2.mean())
print("-" * 50)
print("Best parameters:\n", best_params)
feature_importances

0it [00:00, ?it/s]

/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.024e+11, tolerance: 6.339e+08
  model = cd_fast.enet_coordinate_descent(
/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.933e+11, tolerance: 5.985e+08
  model = cd_fast.enet_coordinate_descent(
/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want 

MAE:
 train    17123.594659
test     18154.145048
dtype: float64
--------------------------------------------------
R2:
 train    0.897390
test     0.882047
dtype: float64
--------------------------------------------------
Best parameters:
 [{'alpha': 1000, 'l1_ratio': 1.0}, {'alpha': 1, 'l1_ratio': 0.9}, {'alpha': 1, 'l1_ratio': 0.9}, {'alpha': 1, 'l1_ratio': 0.9}, {'alpha': 1, 'l1_ratio': 0.9}, {'alpha': 1, 'l1_ratio': 0.9}, {'alpha': 1, 'l1_ratio': 0.9}, {'alpha': 1, 'l1_ratio': 0.9}, {'alpha': 1, 'l1_ratio': 0.9}]


/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.922e+11, tolerance: 7.710e+08
  model = cd_fast.enet_coordinate_descent(
/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.922e+11, tolerance: 7.710e+08
  model = cd_fast.enet_coordinate_descent(


,mean_importance
OverallQual,10415.625850
ExterQual,7037.588259
BedroomAbvGr,6108.349149
KitchenQual,5737.853272
OverallCond,5164.403601
...,...
GrLivArea,7.822476
EnclosedPorch,7.197661
MiscVal,0.770642
LotArea,0.501758


In [138]:
# label with scaling

In [152]:
preprocessor = DataFramePreprocessor(
    numeric_cols=numeric_cols,
    ordinal_cols=ordinal_encode_cols,
    ohe_cols=[],
    label_cols=label_encode_cols,
    ordinal_mappings=ordinal_mappings,
    label_mappings=label_mappings,
    scale_features_except_ohe=True
)
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]}
r2, mae, feature_importances, best_params = run_model(model_class=Lasso, param_grid=param_grid, preprocessor=preprocessor, X=X, y=y)
print("MAE:\n", mae.mean())
print("-" * 50)
print("R2:\n", r2.mean())
print("-" * 50)
print("Best parameters:\n", best_params)
feature_importances

0it [00:00, ?it/s]

/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.208e+09, tolerance: 6.490e+08
  model = cd_fast.enet_coordinate_descent(
/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.582e+09, tolerance: 5.985e+08
  model = cd_fast.enet_coordinate_descent(
/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want 

MAE:
 train    17351.442454
test     19308.888014
dtype: float64
--------------------------------------------------
R2:
 train    0.891875
test     0.869896
dtype: float64
--------------------------------------------------
Best parameters:
 [{'alpha': 1000}, {'alpha': 1000}, {'alpha': 1000}, {'alpha': 1000}, {'alpha': 1000}, {'alpha': 1000}, {'alpha': 1000}, {'alpha': 1000}, {'alpha': 1000}]


,mean_importance
GrLivArea,25904.030800
OverallQual,15413.213177
TotalBsmtSF,8872.317848
BsmtFinSF1,8569.100592
ExterQual,7517.498570
...,...
Condition2,0.000000
Condition1,0.000000
LandSlope,0.000000
LotConfig,0.000000


In [153]:
param_grid = {}
r2, mae, feature_importances, best_params = run_model(model_class=LinearRegression, param_grid=param_grid, preprocessor=preprocessor, X=X, y=y)
print("MAE:\n", mae.mean())
print("-" * 50)
print("R2:\n", r2.mean())
print("-" * 50)
print("Best parameters:\n", best_params)
feature_importances

0it [00:00, ?it/s]

MAE:
 train    1.705323e+04
test     2.091866e+15
dtype: float64
--------------------------------------------------
R2:
 train    9.013837e-01
test    -1.990189e+22
dtype: float64
--------------------------------------------------
Best parameters:
 [{}, {}, {}, {}, {}, {}, {}, {}, {}]


,mean_importance
GrLivArea,1.756623e+16
2ndFlrSF,1.525035e+16
1stFlrSF,1.303654e+16
BsmtUnfSF,1.264200e+16
BsmtFinSF1,1.229143e+16
...,...
YrSold,2.436439e+02
MiscVal,2.420308e+02
Heating,1.986194e+02
MiscFeature,1.939144e+02


In [154]:
param_grid = {
    'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'l1_ratio': [0.1, 0.5, 0.7, 0.9, 1.0]      
}
r2, mae, feature_importances, best_params = run_model(model_class=ElasticNet, param_grid=param_grid, preprocessor=preprocessor, X=X, y=y)
print("MAE:\n", mae.mean())
print("-" * 50)
print("R2:\n", r2.mean())
print("-" * 50)
print("Best parameters:\n", best_params)
feature_importances

0it [00:00, ?it/s]

/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.946e+11, tolerance: 5.985e+08
  model = cd_fast.enet_coordinate_descent(
/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.037e+11, tolerance: 6.339e+08
  model = cd_fast.enet_coordinate_descent(
/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want 

MAE:
 train    17178.045438
test     19545.178831
dtype: float64
--------------------------------------------------
R2:
 train    0.893722
test     0.869004
dtype: float64
--------------------------------------------------
Best parameters:
 [{'alpha': 1000, 'l1_ratio': 1.0}, {'alpha': 1, 'l1_ratio': 0.7}, {'alpha': 1000, 'l1_ratio': 1.0}, {'alpha': 1000, 'l1_ratio': 1.0}, {'alpha': 1, 'l1_ratio': 0.7}, {'alpha': 1, 'l1_ratio': 0.9}, {'alpha': 1, 'l1_ratio': 0.9}, {'alpha': 1, 'l1_ratio': 0.9}, {'alpha': 1, 'l1_ratio': 0.7}]


,mean_importance
GrLivArea,16045.489149
OverallQual,12885.632439
TotalBsmtSF,8112.423023
BsmtFinSF1,7556.026866
MasVnrArea,6680.321022
...,...
MiscFeature,122.239467
MiscVal,122.221164
YrSold,116.122370
LotConfig,93.851785


In [146]:
# ohe with scaling

In [149]:
preprocessor = DataFramePreprocessor(
    numeric_cols=numeric_cols,
    ordinal_cols=ordinal_encode_cols,
    ohe_cols=label_encode_cols,
    label_cols=[],
    ordinal_mappings=ordinal_mappings,
    label_mappings=label_mappings,
    scale_features_except_ohe=True
)
param_grid = {'alpha': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
r2, mae, feature_importances, best_params = run_model(model_class=Lasso, param_grid=param_grid, preprocessor=preprocessor, X=X, y=y)
print("MAE:\n", mae.mean())
print("-" * 50)
print("R2:\n", r2.mean())
print("-" * 50)
print("Best parameters:\n", best_params)
feature_importances

0it [00:00, ?it/s]

/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.377e+11, tolerance: 5.985e+08
  model = cd_fast.enet_coordinate_descent(
/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.169e+11, tolerance: 6.235e+08
  model = cd_fast.enet_coordinate_descent(
/Users/andrewbelyaev/UIUC/STAT 527/STAT527 Final Project/venv/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want 

MAE:
 train    16393.671929
test     19273.208916
dtype: float64
--------------------------------------------------
R2:
 train    0.905832
test     0.870107
dtype: float64
--------------------------------------------------
Best parameters:
 [{'alpha': 100}, {'alpha': 100}, {'alpha': 100}, {'alpha': 100}, {'alpha': 100}, {'alpha': 100}, {'alpha': 100}, {'alpha': 100}, {'alpha': 100}]


,mean_importance
Neighborhood_StoneBr,36384.502076
Neighborhood_NoRidge,31410.977002
Neighborhood_NridgHt,26657.350039
Neighborhood_Crawfor,17310.272836
RoofMatl_WdShngl,15853.270626
...,...
GarageFinish,NaN
Fence,NaN
MiscFeature,NaN
SaleType,NaN


In [150]:
param_grid = {}
r2, mae, feature_importances, best_params = run_model(model_class=LinearRegression, param_grid=param_grid, preprocessor=preprocessor, X=X, y=y)
print("MAE:\n", mae.mean())
print("-" * 50)
print("R2:\n", r2.mean())
print("-" * 50)
print("Best parameters:\n", best_params)
feature_importances

0it [00:00, ?it/s]

MAE:
 train    1.528465e+04
test     1.120542e+13
dtype: float64
--------------------------------------------------
R2:
 train    9.175586e-01
test    -1.969977e+19
dtype: float64
--------------------------------------------------
Best parameters:
 [{}, {}, {}, {}, {}, {}, {}, {}, {}]


,mean_importance
LotShape_IR2,2.005013e+16
LotShape_IR1,2.005013e+16
LotShape_Reg,2.005013e+16
LotShape_IR3,2.005013e+16
SaleCondition_Alloca,1.060400e+16
...,...
GarageFinish,NaN
Fence,NaN
MiscFeature,NaN
SaleType,NaN
